<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/portfolios/optimal_short.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cvxopt

You should consider upgrading via the '/Users/kailiao/PycharmProjects/appcode_draft/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import numpy as np
from cvxopt import matrix as Matrix
from cvxopt.solvers import qp as Solver
from cvxopt.solvers import options as SolverOptions
SolverOptions['show_progress'] = False

#-----------------------------------------------
#
# choice variables are xs=amount to save, xb=amount to borrow, and w=portfolio weights
# constraints are xs>=0, xb>=0, xs + 1'w <= 1, and -xb + 1'w <= 1
# objective function (to minimize) is - rs*xs + rb*xb - means'w
#
#-----------------------------------------------


def optimal(rs, rb, means, cov, raver):
    n = len(means)
    Q = np.zeros((n + 2, n + 2))
    Q[2:, 2:] = raver * cov
    Q = Matrix(Q, tc="d")                               # quadratic form that penalizes variance
    p = np.array([-rs, rb] + list(-np.array(means)))
    p = Matrix(p, (len(p), 1), tc="d")                  # coefficients of objective function
    g = np.array([1, -1] + n * [1])
    a = np.zeros((2, n + 2))
    a[0, 0] = a[1, 1] = -1
    G = Matrix(np.vstack([a, g]))                       # coefficients of constraints
    h = np.array(2 * [0] + [1])
    h = Matrix(h, (len(h), 1), tc="d")                  # right-hand side of constraints
    sol = Solver(Q, p, G, h)
    return sol['x'] if sol['status']=='optimal' else None

# example data

means = np.array([0.05, 0.15, 0.20])  # expected returns
sds = np.array([0.15, 0.25, 0.30])    # standard deviations
c12 = 0.2                             # correlations (1 with 2, 1 with 3, and 2 with 3)
c13 = 0.5
c23 = 0.3
rs = 0.02                             # savings rate
rb = 0.06                             # borrowing rate
raver = 2                             # risk aversion

# solution

corr = np.array([[1,c12,c13], [c12,1,c23], [c13,c23,1]])
cov = np.diag(sds) @ corr @ np.diag(sds)
sol = optimal(rs, rb, means, cov, raver)

if sol:
  sol = list(sol)
  print(f'optimal amount to save is {sol[0]:.1%}')
  print(f'optimal amount to borrow is {sol[1]:.1%}')
  for i in range(len(means)) :
    print(f'optimal investment in asset {i+1} is {sol[i+2]:.1%}')

optimal amount to save is -0.0%
optimal amount to borrow is -0.0%
optimal investment in asset 1 is -67.5%
optimal investment in asset 2 is 72.5%
optimal investment in asset 3 is 95.0%
